In [1]:
import glob
import os
import pandas as pd
import re

In [2]:
import csv
years = ['2007', '2008', '2009','2010', '2011', '2012', '2013', '2014', '2015', '2016']

dict_list={year:{} for year in years}
with open('data/derived/colnames_final.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        final_name = row[-1]
        for year, colname in zip(years, row[:-1]):
            dict_list[year][colname] = final_name

In [3]:
from functools import reduce

dem_files_byYear = []

merge_columns = ['MSA', 'year','MSA_type','Id', 'Id2','Geography']

whitelist_columns = merge_columns + ['Estimate; SEX AND AGE - Median age (years)',
    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)',
    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)'
]


raw_tables_path = 'data/raw/ACS_2007_2016'

# for each year folder, concatenate across the 4 data profile files, 
# save one resulting df per year in the dem_files_byYear list

for year in years:
    # * allows for gziped csv's 
    dem_files = glob.glob(os.path.join(raw_tables_path, year, 'ACS_*_1YR_*with_ann.csv*'))  
    year_pat = re.compile(r'(\d{2})')
    dem_df_list = []
    for fname in dem_files:
        dem_yr_df = pd.read_csv(fname, skiprows=1, header=0)
        dem_yr_df['year'] = int(year)
        dem_yr_df[['MSA', 'MSA_type']] = dem_yr_df['Geography'].str.extract(r'(^.*)\ (M.*)$')
        #rename all columns to a standard naming format
        dem_yr_df.rename_axis(dict_list[year], axis=1, inplace=True)
        #drop any column that isn't a percent or whitelisted
        bad_col_names = [x for x in dem_yr_df.columns.values if 'Percent;' not in x and x not in whitelist_columns]
        dem_yr_df = dem_yr_df[dem_yr_df.columns.drop(bad_col_names)]
        #drop any margin of error columns
        dem_yr_df = dem_yr_df[dem_yr_df.columns.drop(list(dem_yr_df.filter(regex='Margin of Error')))]
        
        
        dem_df_list.append(dem_yr_df)
    dem_year = reduce(lambda x, y: pd.merge(x, y, on = merge_columns), dem_df_list)
    #dem_year.to_csv((folder + 'dataprof.csv'), index=False)
    dem_files_byYear.append(dem_year)
dem_year.shape

/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:26: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


(511, 525)

In [4]:
# Horizontally concatenate each resulting year file so we have one final ACS data file
dem_df = dem_files_byYear[0].copy()
for x in range(1,10):
    dem_df = dem_df.append(dem_files_byYear[x])

print(dem_df.shape)
dem_df.head(3)

(5107, 548)


,Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars),Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars),Estimate; SEX AND AGE - Median age (years),Geography,Id,Id2,MSA,MSA_type,Percent; ANCESTRY - Total population,Percent; ANCESTRY - Total population - American,...,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1940 to 1949,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1950 to 1959,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1960 to 1969,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1970 to 1979,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1980 to 1989,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1990 to 1999,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2000 to 2009,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2010 to 2013,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2014 or later,year
0,52781,42402,39.7,"Aberdeen, WA Micro Area",3100000US10140,10140,"Aberdeen, WA",Micro Area,100,3.6,...,27.4,4.8,6.7,13.3,20.1,12.7,9.6,3.6,1.9,2007
1,51693,39369,34.9,"Abilene, TX Metro Area",3100000US10180,10180,"Abilene, TX",Metro Area,100,6.9,...,9.5,7.0,19.8,11.8,15.5,16.8,9.6,6.5,3.4,2007
2,59453,50240,37.7,"Adrian, MI Micro Area",3100000US10300,10300,"Adrian, MI",Micro Area,100,7.0,...,25.6,5.5,12.7,8.0,14.9,9.1,13.8,8.2,2.2,2007


In [5]:
# save the ACS df
dem_df.to_csv('data/derived/allDem_df.csv', index=False)

# Load and merge with crime data

In [17]:
# load crime data
all_year_dfs = pd.read_csv('data/derived/AC209_RawScraped.csv', index_col=0)
print(all_year_dfs.shape)
all_year_dfs.head(3)

(16262, 15)


,Aggravated assault,Burglary,Counties/principal cities,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,Violent crime,counties,msa_label,msa_pop,year
0,335.0,905.0,City of Abilene,3113.0,251.0,8.0,122523.0,4269.0,70.0,133.0,546.0,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
1,387.0,1104.0,Total area actually reporting,3530.0,298.0,11.0,100.0,4932.0,79.0,137.0,614.0,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
2,227.8,649.9,"Rate per 100,000 inhabitants",2077.9,175.4,6.5,NaN,2903.1,46.5,80.6,361.4,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016


In [18]:
# Filter crime data
norm_rows = all_year_dfs['Counties/principal cities'] == 'Rate per 100,000 inhabitants'
all_year_msa_norm = all_year_dfs[norm_rows]
all_year_msa_norm.head()

,Aggravated assault,Burglary,Counties/principal cities,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,Violent crime,counties,msa_label,msa_pop,year
2,227.8,649.9,"Rate per 100,000 inhabitants",2077.9,175.4,6.5,NaN,2903.1,46.5,80.6,361.4,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
6,121.0,563.3,"Rate per 100,000 inhabitants",1857.3,144.0,6.0,NaN,2564.5,48.9,77.3,253.1,Includes Portage and Summit Counties,"Akron, OH M.S.A.2",703561,2016
10,534.9,949.8,"Rate per 100,000 inhabitants",2676.9,144.2,11.8,NaN,3770.8,28.2,159.9,734.8,"Includes Baker, Dougherty, Lee, Terrell, and W...","Albany, GA M.S.A.",152566,2016
13,59.0,378.6,"Rate per 100,000 inhabitants",2101.1,210.6,0.0,NaN,2690.3,27.9,27.0,113.9,Includes Linn County,"Albany, OR M.S.A.",122030,2016
18,180.0,281.5,"Rate per 100,000 inhabitants",1646.0,82.0,2.0,NaN,2009.5,38.9,68.7,289.7,"Includes Albany, Rensselaer, Saratoga, Schenec...","Albany-Schenectady-Troy, NY M.S.A.",878166,2016


In [19]:
# Clean "M.S.A" and footnotes from scraped data
all_year_msa_norm.loc[:, 'msa_label_strip'] = all_year_msa_norm['msa_label'].str.replace(' M\.S\.A\.[\d\,\ ]*$', '')
# all_year_msa_norm[['msa_label_strip', 'year']].head()

/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
dem_df[['MSA', 'year']].head()

,MSA,year
0,"Aberdeen, WA",2007
1,"Abilene, TX",2007
2,"Adrian, MI",2007
3,"Akron, OH",2007
4,"Albany, GA",2007


Some the MSA labels contain a number after the state abbreviation: e.g. 'Akron, OH2'. This is fixed with a regex replacement like the M.S.A. above.

Be sure that the Year and MSA label match for both the demographics from the ACS and the scraped Crime data.

In [21]:
#some the MSA labels contain a number after the state abbreviation: e.g. 'Akron, OH2'. NEED TO FIX THIS FOR THE MERGE!
all_year_msa_norm.loc[:, 'msa_label_strip'] = all_year_msa_norm['msa_label_strip'].str.replace(r'[\d\,\ ]+$', '').str.strip()
all_year_msa_norm['msa_label_strip'].head(15)

/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2                           Abilene, TX
6                             Akron, OH
10                           Albany, GA
13                           Albany, OR
18          Albany-Schenectady-Troy, NY
21                      Albuquerque, NM
25                       Alexandria, LA
30    Allentown-Bethlehem-Easton, PA-NJ
33                          Altoona, PA
36                         Amarillo, TX
40                             Ames, IA
43                        Anchorage, AK
46                        Ann Arbor, MI
52     Anniston-Oxford-Jacksonville, AL
55                         Appleton, WI
Name: msa_label_strip, dtype: object

In [22]:
all_year_msa_norm[['msa_label_strip', 'year']].head()

,msa_label_strip,year
2,"Abilene, TX",2016
6,"Akron, OH",2016
10,"Albany, GA",2016
13,"Albany, OR",2016
18,"Albany-Schenectady-Troy, NY",2016


In [23]:
dem_df_merged = pd.merge(left=all_year_msa_norm, left_on=['msa_label_strip', 'year'],
                         right=dem_df, right_on=['MSA', 'year'], how='inner')
dem_df_merged.head(10)

,Aggravated assault,Burglary,Counties/principal cities,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,...,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1939 or earlier,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1940 to 1949,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1950 to 1959,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1960 to 1969,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1970 to 1979,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1980 to 1989,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1990 to 1999,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2000 to 2009,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2010 to 2013,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2014 or later
0,227.8,649.9,"Rate per 100,000 inhabitants",2077.9,175.4,6.5,NaN,2903.1,46.5,80.6,...,9.5,8.4,18.6,11.1,15.0,15.5,8.6,9.3,2.6,1.5
1,121.0,563.3,"Rate per 100,000 inhabitants",1857.3,144.0,6.0,NaN,2564.5,48.9,77.3,...,19.0,7.2,14.9,11.6,13.9,9.1,12.9,9.4,1.5,0.6
2,534.9,949.8,"Rate per 100,000 inhabitants",2676.9,144.2,11.8,NaN,3770.8,28.2,159.9,...,6.2,3.2,9.7,14.0,15.2,15.4,17.0,16.9,1.9,0.3
3,59.0,378.6,"Rate per 100,000 inhabitants",2101.1,210.6,0.0,NaN,2690.3,27.9,27.0,...,10.9,8.6,8.7,9.7,20.9,7.9,16.8,12.9,2.5,1.1
4,180.0,281.5,"Rate per 100,000 inhabitants",1646.0,82.0,2.0,NaN,2009.5,38.9,68.7,...,29.8,5.9,9.8,8.9,11.3,12.1,9.6,8.9,2.8,0.9
5,596.7,951.1,"Rate per 100,000 inhabitants",3319.6,1018.0,8.3,NaN,5288.7,57.3,238.7,...,2.8,3.7,9.2,10.0,18.1,16.4,19.0,17.3,2.1,1.3
6,748.5,1271.2,"Rate per 100,000 inhabitants",3111.7,314.9,7.8,NaN,4697.7,50.0,116.9,...,5.2,8.0,15.0,12.4,16.8,12.3,12.6,11.3,5.0,1.3
7,NaN,264.7,"Rate per 100,000 inhabitants",NaN,89.1,2.6,NaN,NaN,19.4,58.3,...,24.8,5.8,11.6,10.2,12.1,10.4,10.7,11.6,2.0,0.9
8,143.3,190.5,"Rate per 100,000 inhabitants",1124.5,44.0,0.8,NaN,1359.0,32.0,36.8,...,31.1,7.0,11.4,11.3,14.9,6.8,8.3,7.5,1.4,0.2
9,413.7,761.3,"Rate per 100,000 inhabitants",2781.3,390.2,5.3,NaN,3932.8,71.4,102.2,...,5.7,6.5,17.4,18.2,14.4,11.6,7.9,12.7,4.7,1.0


In [24]:
dem_df_merged.shape

(3283, 563)

In [25]:
dem_df_merged.head(10)

,Aggravated assault,Burglary,Counties/principal cities,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,...,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1939 or earlier,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1940 to 1949,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1950 to 1959,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1960 to 1969,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1970 to 1979,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1980 to 1989,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 1990 to 1999,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2000 to 2009,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2010 to 2013,Percent; YEAR STRUCTURE BUILT - Total housing units - Built 2014 or later
0,227.8,649.9,"Rate per 100,000 inhabitants",2077.9,175.4,6.5,NaN,2903.1,46.5,80.6,...,9.5,8.4,18.6,11.1,15.0,15.5,8.6,9.3,2.6,1.5
1,121.0,563.3,"Rate per 100,000 inhabitants",1857.3,144.0,6.0,NaN,2564.5,48.9,77.3,...,19.0,7.2,14.9,11.6,13.9,9.1,12.9,9.4,1.5,0.6
2,534.9,949.8,"Rate per 100,000 inhabitants",2676.9,144.2,11.8,NaN,3770.8,28.2,159.9,...,6.2,3.2,9.7,14.0,15.2,15.4,17.0,16.9,1.9,0.3
3,59.0,378.6,"Rate per 100,000 inhabitants",2101.1,210.6,0.0,NaN,2690.3,27.9,27.0,...,10.9,8.6,8.7,9.7,20.9,7.9,16.8,12.9,2.5,1.1
4,180.0,281.5,"Rate per 100,000 inhabitants",1646.0,82.0,2.0,NaN,2009.5,38.9,68.7,...,29.8,5.9,9.8,8.9,11.3,12.1,9.6,8.9,2.8,0.9
5,596.7,951.1,"Rate per 100,000 inhabitants",3319.6,1018.0,8.3,NaN,5288.7,57.3,238.7,...,2.8,3.7,9.2,10.0,18.1,16.4,19.0,17.3,2.1,1.3
6,748.5,1271.2,"Rate per 100,000 inhabitants",3111.7,314.9,7.8,NaN,4697.7,50.0,116.9,...,5.2,8.0,15.0,12.4,16.8,12.3,12.6,11.3,5.0,1.3
7,NaN,264.7,"Rate per 100,000 inhabitants",NaN,89.1,2.6,NaN,NaN,19.4,58.3,...,24.8,5.8,11.6,10.2,12.1,10.4,10.7,11.6,2.0,0.9
8,143.3,190.5,"Rate per 100,000 inhabitants",1124.5,44.0,0.8,NaN,1359.0,32.0,36.8,...,31.1,7.0,11.4,11.3,14.9,6.8,8.3,7.5,1.4,0.2
9,413.7,761.3,"Rate per 100,000 inhabitants",2781.3,390.2,5.3,NaN,3932.8,71.4,102.2,...,5.7,6.5,17.4,18.2,14.4,11.6,7.9,12.7,4.7,1.0


The intermediate outcome of the Demographics Merge is a combined demographics table with all downloaded demographic tables concatenated horizontally (column-wise) each year **allDem_df.csv**.

The primary outcome of the DemographicsMerge is **crime_ACS_merged.csv**, a concatenated sheet with normalized crime statistics, where each row contains data for all crime and demographics for one MSA/year (e.g. 'Abilene TX', '2006')

In [16]:
dem_df_merged.to_csv('data/derived/crime_ACS_merged.csv', index=False)